# **Part one:**
ArcEager with Bi-LSTM

In [ ]:
!pip install datasets  # huggingface library with dataset
!pip install conllu    # aux library for processing CoNLL-U format

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from functools import partial
from datasets import load_dataset

In [ ]:
class ArcEager:

  def __init__(self, sentence):
    self.sentence = sentence
    self.buffer = [i for i in range(len(self.sentence))]
    self.stack = []
    self.arcs = [-1 for _ in range(len(self.sentence))]
    self.shift()

  def shift(self): #like Arc Standard Shift
    b1 = self.buffer[0]
    self.buffer = self.buffer[1:]
    self.stack.append(b1)


  def left_arc(self):

    o1 = self.stack.pop()
    o2 = self.buffer[0]

    # self.arcs[o2] = o1
    self.arcs[o1] = o2

    #self.stack.append(o1)

    # o1 = self.stack.pop()
    # o2 = self.stack.pop()
    # self.arcs[o2] = o1
    # self.stack.append(o1)

  def right_arc(self):

    o1 = self.buffer[0]
    o2 = self.stack.pop()
    self.arcs[o1] = o2
    self.stack.append(o2)
    self.stack.append(o1)
    self.buffer = self.buffer[1:]

    # o1 = self.stack.pop()
    # o2 = self.stack.pop()
    # self.arcs[o1] = o2
    # self.stack.append(o2)


  def reduce(self):
    o1 = self.stack.pop()

  def is_tree_final(self):
    # if len(self.stack) < 3 and len(self.buffer) <2 :
    #     print("is_tree_final", "  stack:  ", len(self.stack) , "  buffer:  ", len(self.buffer))
    # # print(len(self.stack) , " ---- " ,len(self.buffer) )
    return len(self.stack) == 1 and len(self.buffer) == 0


  def print_configuration(self):
    s = [self.sentence[i] for i in self.stack]
    b = [self.sentence[i] for i in self.buffer]
    print(s, b)
    print(self.arcs)

In [ ]:
class Oracle:
  def __init__(self, parser, gold_tree):
    self.parser = parser
    self.gold = gold_tree

  def is_left_arc_gold(self):
    # print("is_left_arc_gold")
    if len(self.parser.buffer) == 0:
      # print("is_left_arc_gold -> inside False")
      return False

    o1 = self.parser.stack[len(self.parser.stack)-1]
    o2 = self.parser.buffer[0]
    # no left arcs on the root
    if self.gold[o1] == o2 and o1 != -1:
      # print("is_left_arc_gold -> inside True")
      return True
    return False


  def is_right_arc_gold(self):
    if len(self.parser.buffer) == 0:
      # print("is_right_arc_gold -> inside False 1")
      return False

    o1 = self.parser.stack[len(self.parser.stack)-1]
    o2 = self.parser.buffer[0]

    if self.gold[o2] != o1:
      # print("is_right_arc_gold -> inside False 2")
      return False

    return True

  def is_shift_gold(self):
    if len(self.parser.buffer) == 0:
      # print("is_shift_gold -> inside False 1")
      return False

    if (self.is_left_arc_gold() or self.is_right_arc_gold() or self.is_reduce_gold()):
      # print("is_shift_gold -> inside False 2")
      return False

    return True

  def is_reduce_gold(self): # check the dependencies
      if len(self.parser.stack) < 2:
          # print("is_reduce_gold -> inside False ")
          return False
      o1 = self.parser.stack[-1]

      if self.parser.arcs[o1] != -1 and all(self.parser.arcs[i] == o1 for i, j in enumerate(self.gold) if j == o1):

          # print("is_reduce_gold -> inside False 2")
          return True
      return False



In [ ]:
sentence = ['<ROOT>','Non', 'soltanto', 'perché', 'oggi', 'come', 'allora', 'la', 'falce', 'è', 'cieca', ',', 'si', 'abbatte', 'indistintamente', 'su', 'uomini', ',', 'donne', 'e', 'bambini', '.']
gold =[-1, 2, 0, 10, 10, 6, 4, 8, 10, 10, 2, 13, 13, 10, 13, 16, 13, 18, 16, 20, 16, 2]

parser = ArcEager(sentence)
oracle = Oracle(parser, gold)


while not parser.is_tree_final():
  if oracle.is_left_arc_gold():
    parser.left_arc()
  elif oracle.is_right_arc_gold():
    parser.right_arc()
  elif oracle.is_reduce_gold():
    parser.reduce()
  elif oracle.is_shift_gold():
    parser.shift()

print("Left Arc: ", oracle.is_left_arc_gold())
print("Right Arc: ", oracle.is_right_arc_gold())
print("Reduce: ", oracle.is_reduce_gold())
print("Shift: ", oracle.is_shift_gold())



print('Parsed:', parser.arcs)
print('Gold:',gold)

Left Arc:  False
Right Arc:  False
Reduce:  False
Shift:  False
Parsed: [-1, 2, 0, 10, 10, 6, 4, 8, 10, 10, 2, 13, 13, 10, 13, 16, 13, 18, 16, 20, 16, 2]
Gold: [-1, 2, 0, 10, 10, 6, 4, 8, 10, 10, 2, 13, 13, 10, 13, 16, 13, 18, 16, 20, 16, 2]


In [ ]:
dataset = load_dataset('universal_dependencies', 'en_lines', split="train")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1032 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1035 [00:00<?, ? examples/s]

Dataset universal_dependencies downloaded and prepared to /root/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7. Subsequent calls will reuse this data.


In [ ]:
# the function returns whether a tree is projective or not. It is currently
# implemented inefficiently by brute checking every pair of arcs.
def is_projective(tree):
  for i in range(len(tree)):
    if tree[i] == -1:
      continue
    left = min(i, tree[i])
    right = max(i, tree[i])

    for j in range(0, left):
      if tree[j] > left and tree[j] < right:
        return False
    for j in range(left+1, right):
      if tree[j] < left or tree[j] > right:
        return False
    for j in range(right+1, len(tree)):
      if tree[j] > left and tree[j] < right:
        return False

  return True

# the function creates a dictionary of word/index pairs: our embeddings vocabulary
# threshold is the minimum number of appearance for a token to be included in the embedding list
def create_dict(dataset, threshold=3):
  dic = {}  # dictionary of word counts
  for sample in dataset:
    for word in sample['tokens']:
      if word in dic:
        dic[word] += 1
      else:
        dic[word] = 1

  map = {}  # dictionary of word/index pairs. This is our embedding list
  map["<pad>"] = 0
  map["<ROOT>"] = 1
  map["<unk>"] = 2 #used for words that do not appear in our list

  next_indx = 3
  for word in dic.keys():
    if dic[word] >= threshold:
      map[word] = next_indx
      next_indx += 1

  return map

In [ ]:
train_dataset = load_dataset('universal_dependencies', 'en_lines', split="train")
dev_dataset = load_dataset('universal_dependencies', 'en_lines', split="validation")
test_dataset = load_dataset('universal_dependencies', 'en_lines', split="test")

# remove non-projective sentences: heads in the gold tree are strings, we convert them to int
train_dataset = [sample for sample in train_dataset if is_projective([-1] + [int(head) for head in sample["head"]])]
# create the embedding dictionary
emb_dictionary = create_dict(train_dataset)

print("Number of samples:")
print("Train:\t", len(train_dataset)) #(train is the number of samples without the non-projective)
print("Dev:\t", len(dev_dataset))
print("Test:\t", len(test_dataset))

Number of samples:
Train:	 2922
Dev:	 1032
Test:	 1035


In [ ]:
def process_sample(sample, get_gold_path = False):

  # put sentence and gold tree in our format
  sentence = ["<ROOT>"] + sample["tokens"]
  gold = [-1] + [int(i) for i in sample["head"]]  #heads in the gold tree are strings, we convert them to int

  # embedding ids of sentence words
  enc_sentence = [emb_dictionary[word] if word in emb_dictionary else emb_dictionary["<unk>"] for word in sentence]

  # gold_path and gold_moves are parallel arrays whose elements refer to parsing steps
  gold_path = []   # record two topmost stack tokens and first buffer token for current step
  gold_moves = []  # contains oracle (canonical) move for current step: 0 is left, 1 right, 2 shift

  if get_gold_path:  # only for training
    parser = ArcEager(sentence)
    oracle = Oracle(parser, gold)

    while not parser.is_tree_final():

      # save configuration
      configuration = [parser.stack[len(parser.stack)-2], parser.stack[len(parser.stack)-1]]
      if len(parser.buffer) == 0:
        configuration.append(-1)
      else:
        configuration.append(parser.buffer[0])
      gold_path.append(configuration)

      # save gold move
      if oracle.is_left_arc_gold():
        gold_moves.append(0)
        parser.left_arc()
      elif oracle.is_right_arc_gold():
        parser.right_arc()
        gold_moves.append(1)
      elif oracle.is_shift_gold():
        parser.shift()
        gold_moves.append(2)
      elif oracle.is_reduce_gold():
        parser.reduce()
        gold_moves.append(3) #************** ***********


  return enc_sentence, gold_path, gold_moves, gold

In [ ]:
def prepare_batch(batch_data, get_gold_path=False):
  data = [process_sample(s, get_gold_path=get_gold_path) for s in batch_data]
  # sentences, paths, moves, trees are parallel arrays, each element refers to a sentence
  sentences = [s[0] for s in data]
  paths = [s[1] for s in data]
  moves = [s[2] for s in data]
  trees = [s[3] for s in data]
  return sentences, paths, moves, trees

In [ ]:
BATCH_SIZE = 8

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=partial(prepare_batch, get_gold_path=True))
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=partial(prepare_batch))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=partial(prepare_batch))

In [ ]:
EMBEDDING_SIZE = 200
LSTM_SIZE = 200
LSTM_LAYERS = 1
MLP_SIZE = 200
DROPOUT = 0.2
EPOCHS = 20
LR = 0.001   # learning rate

In [ ]:
import numpy as np

class Net(nn.Module):

  def __init__(self, device):
    super(Net, self).__init__()
    self.device = device
    self.embeddings = nn.Embedding(len(emb_dictionary), EMBEDDING_SIZE, padding_idx=emb_dictionary["<pad>"])

    # initialize bi-LSTM
    self.lstm = nn.LSTM(EMBEDDING_SIZE, LSTM_SIZE, num_layers = LSTM_LAYERS, bidirectional=True, dropout=DROPOUT)

    # initialize feedforward
    self.w1 = torch.nn.Linear(6*LSTM_SIZE, MLP_SIZE, bias=True)
    self.activation = torch.nn.Tanh()
    self.w2 = torch.nn.Linear(MLP_SIZE, 4, bias=True)
    self.softmax = torch.nn.Softmax(dim=-1)

    self.dropout = torch.nn.Dropout(DROPOUT)


  def forward(self, x, paths):
    # get the embeddings
    x = [self.dropout(self.embeddings(torch.tensor(i).to(self.device))) for i in x]

    # run the bi-lstm
    h = self.lstm_pass(x)

    # for each parser configuration that we need to score we arrange from the
    # output of the bi-lstm the correct input for the feedforward
    mlp_input = self.get_mlp_input(paths, h)

    # run the feedforward and get the scores for each possible action
    out = self.mlp(mlp_input)

    return out

  def lstm_pass(self, x):
    x = torch.nn.utils.rnn.pack_sequence(x, enforce_sorted=False)
    h, (h_0, c_0) = self.lstm(x)
    h, h_sizes = torch.nn.utils.rnn.pad_packed_sequence(h) # size h: (length_sentences, batch, output_hidden_units)
    return h

  def get_mlp_input(self, configurations, h):
    mlp_input = []
    zero_tensor = torch.zeros(2*LSTM_SIZE, requires_grad=False).to(self.device)
    for i in range(len(configurations)): # for every sentence in the batch
      for j in configurations[i]: # for each configuration of a sentence
        mlp_input.append(torch.cat([zero_tensor if j[0]==-1 else h[j[0]][i], zero_tensor if j[1]==-1 else h[j[1]][i], zero_tensor if j[2]==-1 else h[j[2]][i]]))
    mlp_input = torch.stack(mlp_input).to(self.device)
    return mlp_input

  def mlp(self, x):
    return self.softmax(self.w2(self.dropout(self.activation(self.w1(self.dropout(x))))))

  # we use this function at inference time. We run the parser and at each step
  # we pick as next move the one with the highest score assigned by the model
  def infere(self, x):

    parsers = [ArcEager(i) for i in x]

    x = [self.embeddings(torch.tensor(i).to(self.device)) for i in x]

    h = self.lstm_pass(x)

    while not self.parsed_all(parsers):
      # get the current configuration and score next moves
      configurations = self.get_configurations(parsers)
      mlp_input = self.get_mlp_input(configurations, h)
      mlp_out = self.mlp(mlp_input)
      # take the next parsing step
      self.parse_step(parsers, mlp_out)

    # return the predicted dependency tree
    return [parser.arcs for parser in parsers]

  def get_configurations(self, parsers):
    configurations = []

    for parser in parsers:
      if parser.is_tree_final():
        conf = [-1, -1, -1]
      else:
        conf = [parser.stack[len(parser.stack)-2], parser.stack[len(parser.stack)-1]]
        if len(parser.buffer) == 0:
          conf.append(-1)
        else:
          conf.append(parser.buffer[0])
      configurations.append([conf])

    return configurations

  def parsed_all(self, parsers):
    for parser in parsers:
      if not parser.is_tree_final():
        return False
    return True

  # In this function we select and perform the next move according to the scores obtained.
  # We need to be careful and select correct moves, e.g. don't do a shift if the buffer
  # is empty or a left arc if σ2 is the ROOT. For clarity sake we didn't implement
  # these checks in the parser so we must do them here. This renders the function quite ugly
  def parse_step(self, parsers, moves):
        moves_argm = moves.argmax(-1)
        for i in range(len(parsers)):
              if parsers[i].is_tree_final():
                  continue
              else:
                  if moves_argm[i] == 0:
                      stack_len = len(parsers[i].stack)
                      if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                          parsers[i].left_arc()
                      else:
                        if len(parsers[i].stack) >= 2 and len(parsers[i].buffer) > 0:
                          parsers[i].right_arc()
                        elif len(parsers[i].stack) >= 2:
                          parsers[i].reduce()
                        else:
                          parsers[i].shift()
                  elif moves_argm[i] == 1:
                      stack_len = len(parsers[i].stack)
                      if stack_len >= 2 and len(parsers[i].buffer) > 0:
                          parsers[i].right_arc()
                      else:
                          if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                            parsers[i].left_arc()
                          elif len(parsers[i].stack) >= 2:
                            parsers[i].reduce()
                          else:
                            parsers[i].shift()
                  elif moves_argm[i] == 2:
                      if len(parsers[i].buffer) > 0:
                          parsers[i].shift()
                      else:
                        if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                            parsers[i].left_arc()
                        elif len(parsers[i].stack) >= 2 and len(parsers[i].buffer) > 0:
                          parsers[i].right_arc()
                        elif len(parsers[i].stack) >= 2:
                          parsers[i].reduce()
                  elif moves_argm[i] == 3:
                      if len(parsers[i].stack) >= 2:
                          parsers[i].reduce()
                      else:
                        if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                            parsers[i].left_arc()
                        elif len(parsers[i].stack) >= 2 and len(parsers[i].buffer) > 0:
                          parsers[i].right_arc()
                        else:
                          parsers[i].shift()

In [ ]:
def evaluate(gold, preds):
  total = 0
  correct = 0

  for g, p in zip(gold, preds):
    for i in range(1,len(g)):
      total += 1
      if g[i] == p[i]:
        correct += 1

  return correct/total

In [ ]:
def train(model, dataloader, criterion, optimizer):
  model.train()
  total_loss = 0
  count = 0

  for batch in dataloader:
    optimizer.zero_grad()
    sentences, paths, moves, trees = batch

    out = model(sentences, paths)
    labels = torch.tensor(sum(moves, [])).to(device) #sum(moves, []) flatten the array
    loss = criterion(out, labels)

    count +=1
    total_loss += loss.item()

    loss.backward()
    optimizer.step()

  return total_loss/count

def test(model, dataloader):
  model.eval()

  gold = []
  preds = []

  for batch in dataloader:
    sentences, paths, moves, trees = batch
    with torch.no_grad():
      pred = model.infere(sentences)

      gold += trees
      preds += pred

  return evaluate(gold, preds)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
model = Net(device)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)


for epoch in range(EPOCHS):
  avg_train_loss = train(model, train_dataloader, criterion, optimizer)
  val_uas = test(model, dev_dataloader)

  print("Epoch: {:3d} | avg_train_loss: {:5.3f} | dev_uas: {:5.3f} |".format( epoch, avg_train_loss, val_uas))

Device: cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch:   0 | avg_train_loss: 0.997 | dev_uas: 0.577 |
Epoch:   1 | avg_train_loss: 0.924 | dev_uas: 0.614 |
Epoch:   2 | avg_train_loss: 0.902 | dev_uas: 0.649 |
Epoch:   3 | avg_train_loss: 0.887 | dev_uas: 0.656 |
Epoch:   4 | avg_train_loss: 0.874 | dev_uas: 0.659 |
Epoch:   5 | avg_train_loss: 0.865 | dev_uas: 0.665 |
Epoch:   6 | avg_train_loss: 0.857 | dev_uas: 0.668 |
Epoch:   7 | avg_train_loss: 0.851 | dev_uas: 0.676 |
Epoch:   8 | avg_train_loss: 0.846 | dev_uas: 0.684 |
Epoch:   9 | avg_train_loss: 0.840 | dev_uas: 0.680 |
Epoch:  10 | avg_train_loss: 0.837 | dev_uas: 0.690 |
Epoch:  11 | avg_train_loss: 0.834 | dev_uas: 0.688 |
Epoch:  12 | avg_train_loss: 0.830 | dev_uas: 0.691 |
Epoch:  13 | avg_train_loss: 0.810 | dev_uas: 0.693 |
Epoch:  14 | avg_train_loss: 0.797 | dev_uas: 0.700 |
Epoch:  15 | avg_train_loss: 0.794 | dev_uas: 0.695 |
Epoch:  16 | avg_train_loss: 0.793 | dev_uas: 0.696 |
Epoch:  17 | avg_train_loss: 0.790 | dev_uas: 0.699 |
Epoch:  18 | avg_train_loss:

# **Part Two:**
ArcEager with BERT model

In [ ]:
!pip install datasets  # huggingface library with dataset
!pip install conllu    # aux library for processing CoNLL-U format

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from functools import partial
from datasets import load_dataset

In [ ]:
class ArcEager:

  def __init__(self, sentence):
    self.sentence = sentence
    self.buffer = [i for i in range(len(self.sentence))]
    self.stack = []
    self.arcs = [-1 for _ in range(len(self.sentence))]
    self.shift()

  def shift(self): #like Arc Standard Shift
    b1 = self.buffer[0]
    self.buffer = self.buffer[1:]
    self.stack.append(b1)


  def left_arc(self):

    o1 = self.stack.pop()
    o2 = self.buffer[0]

    # self.arcs[o2] = o1
    self.arcs[o1] = o2

    #self.stack.append(o1)

    # o1 = self.stack.pop()
    # o2 = self.stack.pop()
    # self.arcs[o2] = o1
    # self.stack.append(o1)

  def right_arc(self):

    o1 = self.buffer[0]
    o2 = self.stack.pop()
    self.arcs[o1] = o2
    self.stack.append(o2)
    self.stack.append(o1)
    self.buffer = self.buffer[1:]

    # o1 = self.stack.pop()
    # o2 = self.stack.pop()
    # self.arcs[o1] = o2
    # self.stack.append(o2)


  def reduce(self):
    o1 = self.stack.pop()

  def is_tree_final(self):
    # if len(self.stack) < 3 and len(self.buffer) <2 :
    #     print("is_tree_final", "  stack:  ", len(self.stack) , "  buffer:  ", len(self.buffer))
    # # print(len(self.stack) , " ---- " ,len(self.buffer) )
    return len(self.stack) == 1 and len(self.buffer) == 0


  def print_configuration(self):
    s = [self.sentence[i] for i in self.stack]
    b = [self.sentence[i] for i in self.buffer]
    print(s, b)
    print(self.arcs)

In [ ]:
class Oracle:
  def __init__(self, parser, gold_tree):
    self.parser = parser
    self.gold = gold_tree

  def is_left_arc_gold(self):
    # print("is_left_arc_gold")
    if len(self.parser.buffer) == 0:
      # print("is_left_arc_gold -> inside False")
      return False

    o1 = self.parser.stack[len(self.parser.stack)-1]
    o2 = self.parser.buffer[0]
    # no left arcs on the root
    if self.gold[o1] == o2 and o1 != -1:
      # print("is_left_arc_gold -> inside True")
      return True
    return False


  def is_right_arc_gold(self):
    if len(self.parser.buffer) == 0:
      # print("is_right_arc_gold -> inside False 1")
      return False

    o1 = self.parser.stack[len(self.parser.stack)-1]
    o2 = self.parser.buffer[0]

    if self.gold[o2] != o1:
      # print("is_right_arc_gold -> inside False 2")
      return False

    return True

  def is_shift_gold(self):
    if len(self.parser.buffer) == 0:
      # print("is_shift_gold -> inside False 1")
      return False

    if (self.is_left_arc_gold() or self.is_right_arc_gold() or self.is_reduce_gold()):
      # print("is_shift_gold -> inside False 2")
      return False

    return True

  def is_reduce_gold(self): # check the dependencies
      if len(self.parser.stack) < 2:
          # print("is_reduce_gold -> inside False ")
          return False
      o1 = self.parser.stack[-1]

      if self.parser.arcs[o1] != -1 and all(self.parser.arcs[i] == o1 for i, j in enumerate(self.gold) if j == o1):

          # print("is_reduce_gold -> inside False 2")
          return True
      return False



In [ ]:
sentence = ['<ROOT>','Non', 'soltanto', 'perché', 'oggi', 'come', 'allora', 'la', 'falce', 'è', 'cieca', ',', 'si', 'abbatte', 'indistintamente', 'su', 'uomini', ',', 'donne', 'e', 'bambini', '.']
gold =[-1, 2, 0, 10, 10, 6, 4, 8, 10, 10, 2, 13, 13, 10, 13, 16, 13, 18, 16, 20, 16, 2]

parser = ArcEager(sentence)
oracle = Oracle(parser, gold)


while not parser.is_tree_final():
  if oracle.is_left_arc_gold():
    parser.left_arc()
  elif oracle.is_right_arc_gold():
    parser.right_arc()
  elif oracle.is_reduce_gold():
    parser.reduce()
  elif oracle.is_shift_gold():
    parser.shift()

print("Left Arc: ", oracle.is_left_arc_gold())
print("Right Arc: ", oracle.is_right_arc_gold())
print("Reduce: ", oracle.is_reduce_gold())
print("Shift: ", oracle.is_shift_gold())



print('Parsed:', parser.arcs)
print('Gold:  ',gold)

Left Arc:  False
Right Arc:  False
Reduce:  False
Shift:  False
Parsed: [-1, 2, 0, 10, 10, 6, 4, 8, 10, 10, 2, 13, 13, 10, 13, 16, 13, 18, 16, 20, 16, 2]
Gold:   [-1, 2, 0, 10, 10, 6, 4, 8, 10, 10, 2, 13, 13, 10, 13, 16, 13, 18, 16, 20, 16, 2]


In [ ]:
dataset = load_dataset('universal_dependencies', 'en_lines', split="train")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1032 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1035 [00:00<?, ? examples/s]

Dataset universal_dependencies downloaded and prepared to /root/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7. Subsequent calls will reuse this data.


In [ ]:
# the function returns whether a tree is projective or not. It is currently
# implemented inefficiently by brute checking every pair of arcs.
def is_projective(tree):
  for i in range(len(tree)):
    if tree[i] == -1:
      continue
    left = min(i, tree[i])
    right = max(i, tree[i])

    for j in range(0, left):
      if tree[j] > left and tree[j] < right:
        return False
    for j in range(left+1, right):
      if tree[j] < left or tree[j] > right:
        return False
    for j in range(right+1, len(tree)):
      if tree[j] > left and tree[j] < right:
        return False

  return True

# the function creates a dictionary of word/index pairs: our embeddings vocabulary
# threshold is the minimum number of appearance for a token to be included in the embedding list
def create_dict(dataset, threshold=3):
  dic = {}  # dictionary of word counts
  for sample in dataset:
    for word in sample['tokens']:
      if word in dic:
        dic[word] += 1
      else:
        dic[word] = 1

  map = {}  # dictionary of word/index pairs. This is our embedding list
  map["<pad>"] = 0
  map["<ROOT>"] = 1
  map["<unk>"] = 2 #used for words that do not appear in our list

  next_indx = 3
  for word in dic.keys():
    if dic[word] >= threshold:
      map[word] = next_indx
      next_indx += 1

  return map

In [ ]:
train_dataset = load_dataset('universal_dependencies', 'en_lines', split="train")
dev_dataset = load_dataset('universal_dependencies', 'en_lines', split="validation")
test_dataset = load_dataset('universal_dependencies', 'en_lines', split="test")

# remove non-projective sentences: heads in the gold tree are strings, we convert them to int
train_dataset = [sample for sample in train_dataset if is_projective([-1] + [int(head) for head in sample["head"]])]
# create the embedding dictionary
emb_dictionary = create_dict(train_dataset)


In [ ]:
def process_sample_tr(sample, get_gold_path = False):

  # put sentence and gold tree in our format
  sentence = ["<ROOT>"] + sample["tokens"]
  gold = [-1] + [int(i) for i in sample["head"]]  #heads in the gold tree are strings, we convert them to int

  # embedding ids of sentence words
  #enc_sentence = [emb_dictionary[word] if word in emb_dictionary else emb_dictionary["<unk>"] for word in sentence]

  # gold_path and gold_moves are parallel arrays whose elements refer to parsing steps
  gold_path = []   # record two topmost stack tokens and first buffer token for current step
  gold_moves = []  # contains oracle (canonical) move for current step: 0 is left, 1 right, 2 shift

  if get_gold_path:  # only for training
    parser = ArcEager(sentence)
    oracle = Oracle(parser, gold)

    while not parser.is_tree_final():

      # save configuration
      configuration = [parser.stack[len(parser.stack)-2], parser.stack[len(parser.stack)-1]]
      if len(parser.buffer) == 0:
        configuration.append(-1)
      else:
        configuration.append(parser.buffer[0])
      gold_path.append(configuration)

      # save gold move
      if oracle.is_left_arc_gold():
        gold_moves.append(0)
        parser.left_arc()
      elif oracle.is_right_arc_gold():
        parser.right_arc()
        gold_moves.append(1)
      elif oracle.is_shift_gold():
        parser.shift()
        gold_moves.append(2)
      elif oracle.is_reduce_gold():
        parser.reduce()
        gold_moves.append(3) #************** ***********


  return sentence, gold_path, gold_moves, gold

In [ ]:
def prepare_batch(batch,get_gold_path=False):
    data = [process_sample_tr(s, get_gold_path=get_gold_path) for s in batch]
    sentences = [item[0] for item in data]
    paths = [item[1] for item in data]
    moves = [item[2] for item in data]  # Keep as strings
    trees = [item[3] for item in data]

    # Convert sentences, paths, moves, and trees to tensors or perform any other necessary processing

    return sentences, paths, moves, trees


In [ ]:
BATCH_SIZE = 1
                                                            # 200
train_dataloader = torch.utils.data.DataLoader(train_dataset[0:2000], batch_size=BATCH_SIZE, shuffle=True, collate_fn=partial(prepare_batch, get_gold_path=True))
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=partial(prepare_batch))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=partial(prepare_batch))

In [ ]:
EMBEDDING_SIZE = 200
LSTM_SIZE = 200
LSTM_LAYERS = 1
MLP_SIZE = 200
DROPOUT = 0.05
EPOCHS = 20
LR = 0.00001   # learning rate

In [ ]:
!pip install transformers
from transformers import AutoTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
class Net(nn.Module):
    def __init__(self, device):
        super(Net, self).__init__()
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.w1 = torch.nn.Linear(3 * self.bert.config.hidden_size, MLP_SIZE, bias=True)
        self.activation = torch.nn.Tanh()
        self.w2 = torch.nn.Linear(MLP_SIZE, 4, bias=True)
        self.softmax = torch.nn.Softmax(dim=-1)
        self.dropout = torch.nn.Dropout(DROPOUT)

    def forward(self, x, paths):
        x, connector = self.bert_pass(x)
        h = self.lstm_pass(x)
        #print(configurations)

        mlp_input = self.get_mlp_input(paths, x, connector)
        out = self.mlp(mlp_input)
        return out

    def bert_pass(self, x):
        samples = x
        x = self.tokenizer(x, padding=True, truncation=True, is_split_into_words=True, return_tensors='pt')
        word_ids = [x[i].word_ids for i in range(len(x['input_ids']))]
        connector = []
        for j,sentence in enumerate(samples):
          con = []
          con.append(1)
          for i, word in enumerate(sentence):
#             print(word_ids[0])
            con.append( word_ids[j].index(i) )
          connector.append(con)

        input_ids = x['input_ids'].to(self.device)
        attention_mask = x['attention_mask'].to(self.device)
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        h = outputs.last_hidden_state.float()  # Convert h to float
        h = h.permute(1,0,2)
        return h, connector


    def get_mlp_input(self, configurations, h, connector):
        mlp_input = []
        zero_tensor = torch.zeros(768, requires_grad=False).to(self.device)  # Create a zero tensor of the same shape as h[0]

       # print(configurations)

        for i in range(len(configurations)):
            for j in configurations[i]: # for each configuration of a sentence
              if isinstance(j, list):
                  mlp_input.append(torch.cat([zero_tensor if j[0]==-1 else h[ connector[i][ j[0] ] ][i],
                                              zero_tensor if j[1]==-1 else h[ connector[i][ j[1] ] ][i],
                                              zero_tensor if j[2]==-1 else h[ connector[i][ j[2] ] ][i]]))
              else:
                  j = configurations[i]
                  mlp_input.append(torch.cat([zero_tensor if j[0]==-1 else h[ connector[i][ j[0] ] ][i],
                                              zero_tensor if j[1]==-1 else h[ connector[i][ j[1] ] ][i],
                                              zero_tensor if j[2]==-1 else h[ connector[i][ j[2] ] ][i]]))

        return torch.stack(mlp_input).to(self.device)




    def lstm_pass(self, x):
        return x

    def mlp(self, x):
        return self.softmax(self.w2(self.dropout(self.activation(self.w1(self.dropout(x))))))

    def infere(self, x):
          parsers = [ArcEager(i) for i in x]
          h, connector = self.bert_pass(x)

          while not self.parsed_all(parsers):
              configurations = self.get_configurations(parsers)
              mlp_input = self.get_mlp_input(configurations, h, connector)
              mlp_out = self.mlp(mlp_input)
              self.parse_step(parsers, mlp_out)
          return [parser.arcs for parser in parsers]

    def get_configurations(self, parsers):
        configurations = []
        for parser in parsers:
            if parser.is_tree_final():
                conf = [-1, -1, -1]
            else:
                conf = [parser.stack[len(parser.stack)-2], parser.stack[len(parser.stack)-1], -1]
                if len(parser.buffer) > 0:
                    conf[2] = parser.buffer[0]
            configurations.append(conf)
        return configurations


    def parsed_all(self, parsers):
        for parser in parsers:
            if not parser.is_tree_final():
                return False
        return True

    def parse_step(self, parsers, moves):
        moves_argm = moves.argmax(-1)
        for i in range(len(parsers)):
              if parsers[i].is_tree_final():
                  continue
              else:
                  if moves_argm[i] == 0:
                      stack_len = len(parsers[i].stack)
                      if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                          parsers[i].left_arc()
                      else:
                        if len(parsers[i].stack) >= 2 and len(parsers[i].buffer) > 0:
                          parsers[i].right_arc()
                        elif len(parsers[i].stack) >= 2:
                          parsers[i].reduce()
                        else:
                          parsers[i].shift()
                  elif moves_argm[i] == 1:
                      stack_len = len(parsers[i].stack)
                      if stack_len >= 2 and len(parsers[i].buffer) > 0:
                          parsers[i].right_arc()
                      else:
                          if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                            parsers[i].left_arc()
                          elif len(parsers[i].stack) >= 2:
                            parsers[i].reduce()
                          else:
                            parsers[i].shift()
                  elif moves_argm[i] == 2:
                      if len(parsers[i].buffer) > 0:
                          parsers[i].shift()
                      else:
                        if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                            parsers[i].left_arc()
                        elif len(parsers[i].stack) >= 2 and len(parsers[i].buffer) > 0:
                          parsers[i].right_arc()
                        elif len(parsers[i].stack) >= 2:
                          parsers[i].reduce()
                  elif moves_argm[i] == 3:
                      if len(parsers[i].stack) >= 2:
                          parsers[i].reduce()
                      else:
                        if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                            parsers[i].left_arc()
                        elif len(parsers[i].stack) >= 2 and len(parsers[i].buffer) > 0:
                          parsers[i].right_arc()
                        else:
                          parsers[i].shift()


In [ ]:
def evaluate(gold, preds):
  total = 0
  correct = 0

  for g, p in zip(gold, preds):
    for i in range(1,len(g)):
      total += 1
      if g[i] == p[i]:
        correct += 1

  return correct/total

In [ ]:
def train(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0
    count = 0

    for batch in dataloader:
        optimizer.zero_grad()
        sentences, paths, moves, trees = batch

        out = model(sentences, paths)  # Pass sentences and paths to the model
        labels = torch.tensor(sum(moves, [])).to(device)
        loss = criterion(out, labels)

        count += 1
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    return total_loss / count


def test(model, dataloader):
    model.eval()

    gold = []
    preds = []

    for batch in dataloader:
        sentences, paths, moves, trees = batch
        with torch.no_grad():
            pred = model.infere(sentences)  # Pass sentences to the model

        gold += trees
        preds += pred

    return evaluate(gold, preds)


In [ ]:
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
from transformers import BertModel, BertTokenizer

model = Net(device)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

Device: cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for epoch in range(EPOCHS):
    avg_train_loss = train(model, train_dataloader, criterion, optimizer)
    val_uas = test(model, dev_dataloader)

    print("Epoch: {:3d} | avg_train_loss: {:5.3f} | dev_uas: {:5.3f} |".format(epoch, avg_train_loss, val_uas))

Epoch:   0 | avg_train_loss: 1.018 | dev_uas: 0.621 |
Epoch:   1 | avg_train_loss: 0.887 | dev_uas: 0.675 |
Epoch:   2 | avg_train_loss: 0.854 | dev_uas: 0.709 |
Epoch:   3 | avg_train_loss: 0.837 | dev_uas: 0.753 |
Epoch:   4 | avg_train_loss: 0.826 | dev_uas: 0.771 |
Epoch:   5 | avg_train_loss: 0.818 | dev_uas: 0.766 |
Epoch:   6 | avg_train_loss: 0.813 | dev_uas: 0.798 |
Epoch:   7 | avg_train_loss: 0.808 | dev_uas: 0.784 |
Epoch:   8 | avg_train_loss: 0.804 | dev_uas: 0.788 |
Epoch:   9 | avg_train_loss: 0.802 | dev_uas: 0.805 |
Epoch:  10 | avg_train_loss: 0.801 | dev_uas: 0.809 |
Epoch:  11 | avg_train_loss: 0.799 | dev_uas: 0.809 |
Epoch:  12 | avg_train_loss: 0.797 | dev_uas: 0.807 |
Epoch:  13 | avg_train_loss: 0.796 | dev_uas: 0.822 |
Epoch:  14 | avg_train_loss: 0.793 | dev_uas: 0.822 |
Epoch:  15 | avg_train_loss: 0.793 | dev_uas: 0.795 |
Epoch:  16 | avg_train_loss: 0.793 | dev_uas: 0.811 |
Epoch:  17 | avg_train_loss: 0.791 | dev_uas: 0.808 |
Epoch:  18 | avg_train_loss: